# LCA with only background

### Preparation

Before use the library, you have to direct to the right brightway project and make sure you have the database.

In [1]:
import bw2data as bd

bd.projects.set_current("advlca25")
bd.databases

Databases dictionary with 2 object(s):
	ecoinvent-3.11-biosphere
	ecoinvent-3.11-consequential

In [2]:
import os
import sys
sys.path.append(os.path.abspath(".."))
from bw_bamboo.background_importer import *
from bw_bamboo.foreground_importer import *
from bw_bamboo.datapackage_builder import *
from bw_bamboo.uncertainty_handler import *
from bw_bamboo.lca_wrapper import *

0. Define some constants for this notebook

In [3]:
METHOD = ('IPCC 2013', 'climate change', 'global warming potential (GWP100)')
FG_EMISSION_FILE = os.path.join(os.getcwd(), "data", "EXIOBASE-ecoinvent-bio-bw-GHG.csv")

EXIOBASE_AGGREGATED_A_FILE = os.path.join(os.getcwd(), "data/", "A.txt")
EXIOBASE_AGGREGATED_S_FILE = os.path.join(os.getcwd(), "data/", "S.txt")
GHG = ["CO2 - combustion - air",
        "CO2 - non combustion - Cement production - air",
        "CO2 - non combustion - Lime production - air",
        "CO2 - waste - biogenic - air", 
        "CO2 - waste - fossil - air",
        "CO2 - agriculture - peat decay - air", 
        "CH4 - agriculture - air",
        "CH4 - waste - air",
        "CH4 - combustion - air",
        "CH4 - non combustion - Extraction/production of (natural) gas - air",
        "CH4 - non combustion - Extraction/production of crude oil - air",
        "CH4 - non combustion - Mining of antracite - air",
        "CH4 - non combustion - Mining of bituminous coal - air",
        "CH4 - non combustion - Mining of coking coal - air",
        "CH4 - non combustion - Mining of lignite (brown coal) - air",
        "CH4 - non combustion - Mining of sub-bituminous coal - air",
        "CH4 - non combustion - Oil refinery - air",
        "N2O - combustion - air",
        "N2O - agriculture - air",
        "SF6 - air"]

1. Get technosphere matrix

In [4]:
bg_importer = BackgroundImporter()

tech_df = pd.read_table(EXIOBASE_AGGREGATED_A_FILE, sep='\t', header=None, low_memory=False)
raw_tech = tech_df.iloc[3:, 2:].astype('float').to_numpy()
tech_matrix = bg_importer.form_tech_matrix(raw_tech)

2. Get biosphere matrix

In [5]:
bio_df = pd.read_csv(EXIOBASE_AGGREGATED_S_FILE, header=[0,1], index_col=[0], sep='\t', low_memory=False)
bio_matrix = bg_importer.form_bio_matrix(bio_df, GHG)

3. Get characterization factor matrix

In [ ]:
cf_matrix = bg_importer.form_cf_matrix("ecoinvent-3.11-biosphere", METHOD, FG_EMISSION_FILE, GHG)
cf_matrix.diagonal()  # print the diagnal for checking
# cf_matrix = [1., 1., 1., 1., 1., 1., 27.0, 29.8, 29.8, 29.8, 29.8, 29.8, 29.8, 29.8, 29.8, 29.8, 29.8, 273., 273., 25200.]
# cf_matrix = np.diagflat(cf_matrix)

Characterization factor data imcomplete, missing: ['28e1e2d6-97ad-4dfd-932a-9edad36dcab9', 'eba59fd6-f37e-41dc-9ca3-c7ea22d602c7']


4. Build datapackage

In [7]:
# check the shape of all matrix
print(tech_matrix.shape)
print(bio_matrix.shape)
print(cf_matrix.shape)

(76, 76)
(20, 76)
(20, 20)


In [8]:
dp_builder = DatapackageBuilder()
datapackage_data = dp_builder.prepare_bw_matrix(tech_matrix, bio_matrix, cf_matrix)
datapackage = dp_builder.prepare_datapackage(datapackage_data)

5. Run static simulation through brightway directly

In [9]:
import bw2calc as bc

index = 68
activity_name = "RoW-Services"

lca = bc.LCA(
    demand={index: 1},
    data_objs=[datapackage],
)
lca.lci()
lca.lcia()

print(f"{activity_name}: {lca.score}")

RoW-Services: 291685.0667897694


6. Compare with lca manually.

In [10]:
lca_wrapper = LCAWrapper()

manual_tech_Data = -tech_matrix
np.fill_diagonal(manual_tech_Data, -manual_tech_Data.diagonal())

manual_lca = lca_wrapper.manual_lca(manual_tech_Data, bio_matrix, cf_matrix, 68)

print(f"{activity_name}: {manual_lca}")


RoW-Services: 291685.06678976933
